In [ ]:
import pandas as pd
import plotly.express as px

# 1. Caricamento del dataset
file_path = './datasets/ex_smokers.csv'
df = pd.read_csv(file_path)

# Pulizia e conversione
df['TIME_PERIOD'] = pd.to_numeric(df['TIME_PERIOD'], errors='coerce')
df['Osservazione'] = pd.to_numeric(df['Osservazione'], errors='coerce')
df = df.dropna(subset=['TIME_PERIOD', 'Osservazione'])

# 2. Aggregazione in 3 macro-categorie d'età
# Mappiamo le micro-fasce ISTAT in 3 gruppi logici
age_mapping = {
    'Y14-17': '14-34 anni', 'Y18-19': '14-34 anni', 'Y20-24': '14-34 anni', 'Y25-34': '14-34 anni',
    'Y35-44': '35-64 anni', 'Y45-54': '35-64 anni', 'Y55-59': '35-64 anni', 'Y60-64': '35-64 anni',
    'Y65-74': '65+ anni', 'Y_GE75': '65+ anni'
}
df['Age_Group_Agg'] = df['AGE'].map(age_mapping)

# Definizione tappe legislative (Milestones)
milestones = {
    2005: "Legge Sirchia<br>(Chiuso)",
    2016: "Direttiva UE<br>(Immagini Shock)",
    2021: "Divieti aperto<br>(10m)"
}

def apply_layout_style(fig, title, y_label):
    """Applica lo stile uniforme e le linee delle leggi"""
    for year, label in milestones.items():
        fig.add_vline(x=year, line_dash="dash", line_color="red", opacity=0.4)
        fig.add_annotation(
            x=year, y=1.05, yref="paper", text=label, showarrow=False,
            font=dict(color="red", size=10, family="Arial Black"),
            bgcolor="white", opacity=0.8
        )
    fig.update_layout(
        title=f"<b>{title}</b>",
        font=dict(family="Arial", size=14),
        hovermode="x unified",
        plot_bgcolor='white',
        xaxis=dict(showgrid=False, tickangle=45),
        yaxis=dict(title=y_label, showgrid=True, gridcolor='LightGray'),
        margin=dict(t=120, b=50, l=50, r=50)
    )

# --- GRAFICO 1: TREND TEMPORALE (3 CATEGORIE) ---
# Filtriamo: Totale (SEX=9), Valori in Migliaia (THV), e solo le righe mappate nelle 3 categorie
df_3_age = df[
    (df['SEX'] == 9) &
    (df['MEASURE'] == 'THV') &
    (df['Age_Group_Agg'].notna())
].groupby(['TIME_PERIOD', 'Age_Group_Agg'])['Osservazione'].sum().reset_index()

fig1 = px.line(
    df_3_age, x='TIME_PERIOD', y='Osservazione', color='Age_Group_Agg', markers=True,
    labels={'TIME_PERIOD': 'Anno', 'Osservazione': 'Ex Fumatori (Migliaia)', 'Age_Group_Agg': 'Fascia d\'Età'}
)
apply_layout_style(fig1, "Andamento Ex Fumatori per Macro-Fasce d'Età", "Migliaia di persone")
fig1.show()

# --- GRAFICO 2: BARRE DISTRIBUZIONE (ULTIMO ANNO) ---
last_year = df_3_age['TIME_PERIOD'].max()
df_last_3 = df_3_age[df_3_age['TIME_PERIOD'] == last_year].sort_values('Osservazione', ascending=False)

fig2 = px.bar(
    df_last_3, x='Age_Group_Agg', y='Osservazione', color='Age_Group_Agg',
    labels={'Osservazione': 'Ex Fumatori (Migliaia)', 'Age_Group_Agg': 'Fascia d\'Età'},
    title=f"<b>Confronto Volumi Ex Fumatori nel {last_year}</b>",
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig2.update_layout(plot_bgcolor='white', showlegend=False, yaxis_title="Migliaia di persone")
fig2.show()

